In [1]:
import random
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import os
import keras
import glob
from sklearn.preprocessing import MinMaxScaler

In [2]:
files=glob.glob("GA Population/*.xlsx",recursive=True)
population=[]
for i in range(0, len(files)):
    parent=pd.read_excel(files[i],header=None).values
    population.append(parent)
population=np.array(population,dtype=object)

In [3]:
all_geometry = []
all_results = []
for i in range(0, population.shape[0]):
    pop = pd.DataFrame(population[i])
    pop = pop.drop(index=[20, 21], axis=0)
    geometry = pop.iloc[14:22, 1:4].values
    results = pop.iloc[82:96, 2:6].values
    all_geometry.append(geometry)
    all_results.append(results)
all_geometry = np.array(all_geometry)
all_results = np.array(all_results)

In [4]:
3%2

1

In [5]:
def fitness_function(pop,val):
    fitness=[0]*pop.shape[0]

    for i in range(pop.shape[0]):
        
        if (val[i][0][0][0]>=-2 and val[i][0][0][0]<=2) and (val[i][1][0][0]>=-2 and val[i][1][0][0]<=2):
            camber_fitness=abs(val[i][1][0][0]-val[i][0][0][0])/5
            fitness[i]+=camber_fitness
        else:
            fitness[i]=99999
            continue
            
        if (val[i][0][0][1]>=-1 and val[i][0][0][1]<=1) and (val[i][1][0][1]>=-1 and val[i][1][0][1]<=1):
            toe_fitness=abs(val[i][1][0][1]-val[i][0][0][1])/4
            fitness[i]+=toe_fitness
        else:
            fitness[i]=99999
            continue
        
        if (val[i][0][0][2]>=4 and val[i][0][0][2]<=9) and (val[i][1][0][2]>=4 and val[i][1][0][2]<=9):
            fitness[i]+=abs(val[i][1][0][2]-val[i][0][0][2])/2
        else:
            fitness[i]=99999
            continue
            
        if (val[i][0][0][3]>=3 and val[i][0][0][3]<=8) and (val[i][1][0][3]>=3 and val[i][1][0][3]<=8):
            fitness[i]+=abs(val[i][1][0][3]-val[i][0][0][3])/2
        else:
            fitness[i]=99999
            continue

    return fitness

In [6]:
#Use tournament selection

In [7]:
def roulette_selection(population,no_parents,fitness):
    # Roulette_wheel_selection 
    previous_prob=0
    fitness=[1/(x+1e-4) for x in fitness]
    prob_sel=[x/sum(fitness) for x in fitness]
    prob=[]
    parents=np.empty([no_parents,population.shape[1],population.shape[2]])
    for i in range(len(prob_sel)):
        previous_prob=previous_prob+prob_sel[i]
        prob.append(previous_prob)
    
    for i in range(no_parents):
        n=random.random()
        for j in range(len(prob)):
            if n<prob[j]:
                parents[i]=population[j]
                break
        
    return parents

In [8]:
def sus_selection(population,no_parents,fitness):
    pass

In [9]:
def crossover(parent,no_offspring,alpha):
    
    offspring=np.empty([no_offspring,parent.shape[1],parent.shape[2]])
    
    for i in range(no_offspring):
        parent1_index=i%parent.shape[0]
        parent2_index=(i+1)%parent.shape[0]
        n=random.random()
        offspring[i]=n*parent[parent1_index]+(1-n)*parent[parent2_index]
        
    return offspring

In [15]:
# creed mutation
def mutation(offspring,mutation_rate):
    
    mutation=np.array(offspring,copy=True)
        
    if random.randint(1,5)%4 == 0 :
        k = random.randint(0,offspring.shape[0]-1)
        j = random.randint(0,offspring.shape[1]-1)
        i = random.randint(0,offspring.shape[2]-1)
        mutation[k][j][i]=random.randint(int(all_geometry[:,j,i].min()),int(all_geometry[:,j,i].max()))+random.random()

    
    return mutation

In [16]:
def prepare_dataset(pop,model):
    X_flat=pop.reshape((pop.shape[0], -1))
    X=[]
    X_final=[]
    bump_val=[200,-60]
        
    for i in range(X_flat.shape[0]):
        for j in range(len(bump_val)):
            X.append(list(np.append(X_flat[i],bump_val[j])))
        #X_final.append(X)
        #X=[]
            
    X_final=np.array(X)
        
    train_df=pd.DataFrame(X_final)
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_train = scaler.fit_transform(train_df)
    scaled_train_df = pd.DataFrame(scaled_train, columns=train_df.columns.values)
        
    val=[]
    val_total=[]
    X = scaled_train_df[scaled_train_df.columns[:25]].values
    count=0
    for i in range(X.shape[0]):
        count+=1
        val1=model.predict(X[i].reshape(1,X[i].shape[0]))
        val.append(val1)
        if count==2:
            val_total.append(val)
            val=[]
            count=0
    return val_total

In [17]:
model=keras.models.load_model('GA_ANN_normal.h5')

In [18]:
def optimization(pop,no_generation,no_parent,mutation_rate):
    
    for m in range(no_generation):

        val_total=prepare_dataset(pop,model)
        pop_size=38
        no_offspring=pop_size-no_parent
        fitness=fitness_function(pop,val_total)
        parent=roulette_selection(pop,no_parent,fitness)
        offspring=crossover(parent,no_offspring,0.7)
        mutate=mutation(offspring,mutation_rate)
        pop=np.concatenate((pop,mutate),axis=0)
        
    return pop,fitness

In [19]:
pop,fitness=optimization(all_geometry,10,6,0.15)

In [20]:
index=fitness.index(min(fitness))

In [21]:
index

0

In [22]:
fitness[index]

99999

In [23]:
pop[index]

array([[3819, -143.61, 228.93],
       [4179, -128.14, 175.28],
       [4092, -538.77, 175.49],
       [4092.5, -147.9, 447.28],
       [4332, -129.68, 434.68],
       [4092.5, -496.74, 443.08],
       [4214.5, -531.77, 216.12],
       [4245.5, -179.73, 229.96]], dtype=object)

In [24]:
fitness

[99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,
 99999,


In [25]:
valT = prepare_dataset(pop,model)

In [26]:
valT[0]

[array([[10.232421,  2.830523, 25.952757, -9.120455]], dtype=float32),
 array([[-0.23079108, -6.629354  , -2.0559406 ,  6.1220336 ]],
       dtype=float32)]

In [27]:
pop[0]

array([[3819, -143.61, 228.93],
       [4179, -128.14, 175.28],
       [4092, -538.77, 175.49],
       [4092.5, -147.9, 447.28],
       [4332, -129.68, 434.68],
       [4092.5, -496.74, 443.08],
       [4214.5, -531.77, 216.12],
       [4245.5, -179.73, 229.96]], dtype=object)

NameError: name 'offspring' is not defined